In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
970587,2021-02-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
970588,2021-02-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
970589,2021-02-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
970590,2021-02-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
26483,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26485,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26487,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26489,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26491,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
602701,2021-02-05,Arkansas,Arkansas,5001,1855,36.00,5000,Arkansas,AR,Arkansas,State,3017804
602703,2021-02-06,Arkansas,Arkansas,5001,1870,37.00,5000,Arkansas,AR,Arkansas,State,3017804
602705,2021-02-07,Arkansas,Arkansas,5001,1881,37.00,5000,Arkansas,AR,Arkansas,State,3017804
602707,2021-02-08,Arkansas,Arkansas,5001,1884,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
970587,2021-02-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
970588,2021-02-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
970589,2021-02-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
970590,2021-02-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-09') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
385,2021-02-09,Snohomish,Washington,29489,504.00,53061,WA,County,822083,3587.11,61.31
768,2021-02-09,Cook,Illinois,461757,9585.00,17031,IL,County,5150233,8965.75,186.11
1150,2021-02-09,Orange,California,253900,3416.00,6059,CA,County,3175692,7995.11,107.57
1531,2021-02-09,Maricopa,Arizona,492492,8125.00,4013,AZ,County,4485414,10979.86,181.14
1912,2021-02-09,Los Angeles,California,1152740,18360.00,6037,CA,County,10039107,11482.50,182.88
...,...,...,...,...,...,...,...,...,...,...,...
970235,2021-02-09,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
970355,2021-02-09,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
970444,2021-02-09,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
970529,2021-02-09,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
385,2021-02-09,Snohomish,Washington,29489,504.00,53061,WA,County,822083,3587.11,61.31,61.31,3587.11
768,2021-02-09,Cook,Illinois,461757,9585.00,17031,IL,County,5150233,8965.75,186.11,186.11,8965.75
1150,2021-02-09,Orange,California,253900,3416.00,6059,CA,County,3175692,7995.11,107.57,107.57,7995.11
1531,2021-02-09,Maricopa,Arizona,492492,8125.00,4013,AZ,County,4485414,10979.86,181.14,181.14,10979.86
1912,2021-02-09,Los Angeles,California,1152740,18360.00,6037,CA,County,10039107,11482.50,182.88,182.88,11482.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
970235,2021-02-09,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
970355,2021-02-09,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
970444,2021-02-09,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
970529,2021-02-09,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
685449,2021-02-09,Clarke,Alabama,3346,42.00,1025,AL,County,23622,14164.76,177.80,177.80,14164.76,1
630054,2021-02-09,Hale,Alabama,1998,56.00,1065,AL,County,14651,13637.29,382.23,382.23,13637.29,2
493581,2021-02-09,Lowndes,Alabama,1278,42.00,1085,AL,County,9726,13140.04,431.83,431.83,13140.04,3
437881,2021-02-09,Etowah,Alabama,12837,273.00,1055,AL,County,102268,12552.31,266.95,266.95,12552.31,4
367716,2021-02-09,Franklin,Alabama,3923,69.00,1059,AL,County,31362,12508.77,220.01,220.01,12508.77,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601436,2021-02-09,Converse,Wyoming,926,15.00,56009,WY,County,13822,6699.46,108.52,108.52,6699.46,19
923964,2021-02-09,Platte,Wyoming,562,11.00,56031,WY,County,8393,6696.06,131.06,131.06,6696.06,20
830197,2021-02-09,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
746681,2021-02-09,Lincoln,Wyoming,1166,11.00,56023,WY,County,19830,5879.98,55.47,55.47,5879.98,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
493581,2021-02-09,Lowndes,Alabama,1278,42.00,1085,AL,County,9726,13140.04,431.83,431.83,13140.04,3,1
436593,2021-02-09,Clay,Alabama,1371,52.00,1027,AL,County,13235,10358.90,392.90,392.90,10358.90,24,2
630054,2021-02-09,Hale,Alabama,1998,56.00,1065,AL,County,14651,13637.29,382.23,382.23,13637.29,2,3
532098,2021-02-09,Greene,Alabama,828,29.00,1063,AL,County,8111,10208.36,357.54,357.54,10208.36,28,4
492939,2021-02-09,Crenshaw,Alabama,1368,49.00,1041,AL,County,13772,9933.20,355.79,355.79,9933.20,31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530817,2021-02-09,Hot Springs,Wyoming,357,3.00,56017,WY,County,4413,8089.73,67.98,67.98,8089.73,15,19
728214,2021-02-09,Uinta,Wyoming,1994,12.00,56041,WY,County,20226,9858.60,59.33,59.33,9858.60,5,20
746681,2021-02-09,Lincoln,Wyoming,1166,11.00,56023,WY,County,19830,5879.98,55.47,55.47,5879.98,22,21
209646,2021-02-09,Teton,Wyoming,3266,9.00,56039,WY,County,23464,13919.20,38.36,38.36,13919.20,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,61.31,3587.11,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,61.31,3587.11,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,61.31,3587.11,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,61.31,3587.11,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,61.31,3587.11,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968291,2021-02-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
968292,2021-02-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
968293,2021-02-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
968294,2021-02-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
682839,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,177.80,14164.76,50,1,2.00
682840,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,177.80,14164.76,50,1,0.00
682841,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,177.80,14164.76,50,1,1.00
682842,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,177.80,14164.76,50,1,4.00
682843,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,177.80,14164.76,50,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759045,2021-02-05,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,1.00
759046,2021-02-06,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,0.00
759047,2021-02-07,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,0.00
759048,2021-02-08,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
491603,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13140.04,1,3,1.00,0.00
491604,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13140.04,1,3,0.00,0.00
491605,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13140.04,1,3,0.00,0.00
491606,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13140.04,1,3,0.00,0.00
491607,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13140.04,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489993,2021-02-05,Albany,Wyoming,3818,11.00,56001,WY,County,38880,9819.96,28.29,28.29,9855.97,23,6,3.00,0.00
489994,2021-02-06,Albany,Wyoming,3823,11.00,56001,WY,County,38880,9832.82,28.29,28.29,9855.97,23,6,5.00,0.00
489995,2021-02-07,Albany,Wyoming,3825,11.00,56001,WY,County,38880,9837.96,28.29,28.29,9855.97,23,6,2.00,0.00
489996,2021-02-08,Albany,Wyoming,3831,11.00,56001,WY,County,38880,9853.40,28.29,28.29,9855.97,23,6,6.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-09') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
251270,2021-02-09,Imperial,California,26526,572.00,6025,CA,County,181215,14637.86,315.65,315.65,14637.86,1,2,8.00,12.00
1912,2021-02-09,Los Angeles,California,1152740,18360.00,6037,CA,County,10039107,11482.50,182.88,182.88,11482.50,2,5,3175.00,225.00
439121,2021-02-09,Inyo,California,1126,32.00,6027,CA,County,18039,6242.03,177.39,177.39,6242.03,3,32,2.00,3.00
56241,2021-02-09,Stanislaus,California,53278,874.00,6099,CA,County,550660,9675.30,158.72,158.72,9675.30,4,10,282.00,7.00
69678,2021-02-09,Tulare,California,45617,651.00,6107,CA,County,466195,9784.96,139.64,139.64,9784.96,5,9,97.00,6.00
26142,2021-02-09,Riverside,California,283525,3414.00,6065,CA,County,2470546,11476.21,138.19,138.19,11476.21,6,6,789.00,37.00
398117,2021-02-09,Merced,California,27509,367.00,6047,CA,County,277680,9906.73,132.17,132.17,9906.73,7,8,85.00,0.00
532040,2021-02-09,Kings,California,21342,199.00,6031,CA,County,152940,13954.49,130.12,130.12,13954.49,8,3,45.00,2.00
25462,2021-02-09,Fresno,California,91430,1254.00,6019,CA,County,999101,9151.23,125.51,125.51,9151.23,9,14,246.00,30.00
44785,2021-02-09,San Joaquin,California,64396,929.00,6077,CA,County,762148,8449.28,121.89,121.89,8449.28,10,18,258.00,6.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
923515,2021-02-09,Lassen,California,5492,18.00,6035,CA,County,30573,17963.56,58.88,58.88,17963.56,34,1,3.00,0.00
251270,2021-02-09,Imperial,California,26526,572.00,6025,CA,County,181215,14637.86,315.65,315.65,14637.86,1,2,8.00,12.00
532040,2021-02-09,Kings,California,21342,199.00,6031,CA,County,152940,13954.49,130.12,130.12,13954.49,8,3,45.00,2.00
125432,2021-02-09,San Bernardino,California,279412,2142.00,6071,CA,County,2180085,12816.56,98.25,98.25,12816.56,13,4,610.00,1.00
1912,2021-02-09,Los Angeles,California,1152740,18360.00,6037,CA,County,10039107,11482.50,182.88,182.88,11482.50,2,5,3175.00,225.00
26142,2021-02-09,Riverside,California,283525,3414.00,6065,CA,County,2470546,11476.21,138.19,138.19,11476.21,6,6,789.00,37.00
157581,2021-02-09,Kern,California,98187,660.00,6029,CA,County,900202,10907.22,73.32,73.32,10907.22,27,7,665.00,1.00
398117,2021-02-09,Merced,California,27509,367.00,6047,CA,County,277680,9906.73,132.17,132.17,9906.73,7,8,85.00,0.00
69678,2021-02-09,Tulare,California,45617,651.00,6107,CA,County,466195,9784.96,139.64,139.64,9784.96,5,9,97.00,6.00
56241,2021-02-09,Stanislaus,California,53278,874.00,6099,CA,County,550660,9675.30,158.72,158.72,9675.30,4,10,282.00,7.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
251270,2021-02-09,Imperial,California,26526,572.00,6025,CA,County,181215,14637.86,315.65,315.65,14637.86,1,2,8.00,12.00
1912,2021-02-09,Los Angeles,California,1152740,18360.00,6037,CA,County,10039107,11482.50,182.88,182.88,11482.50,2,5,3175.00,225.00
439121,2021-02-09,Inyo,California,1126,32.00,6027,CA,County,18039,6242.03,177.39,177.39,6242.03,3,32,2.00,3.00
56241,2021-02-09,Stanislaus,California,53278,874.00,6099,CA,County,550660,9675.30,158.72,158.72,9675.30,4,10,282.00,7.00
69678,2021-02-09,Tulare,California,45617,651.00,6107,CA,County,466195,9784.96,139.64,139.64,9784.96,5,9,97.00,6.00
26142,2021-02-09,Riverside,California,283525,3414.00,6065,CA,County,2470546,11476.21,138.19,138.19,11476.21,6,6,789.00,37.00
398117,2021-02-09,Merced,California,27509,367.00,6047,CA,County,277680,9906.73,132.17,132.17,9906.73,7,8,85.00,0.00
532040,2021-02-09,Kings,California,21342,199.00,6031,CA,County,152940,13954.49,130.12,130.12,13954.49,8,3,45.00,2.00
25462,2021-02-09,Fresno,California,91430,1254.00,6019,CA,County,999101,9151.23,125.51,125.51,9151.23,9,14,246.00,30.00
44785,2021-02-09,San Joaquin,California,64396,929.00,6077,CA,County,762148,8449.28,121.89,121.89,8449.28,10,18,258.00,6.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,315.65,14637.86,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,315.65,14637.86,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,315.65,14637.86,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,315.65,14637.86,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,315.65,14637.86,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3282,02/05/21,Lassen,California,5435,17.00,6035,CA,County,30573,17777.12,55.60,58.88,17963.56,34,1,0.00,0.00
3283,02/06/21,Lassen,California,5435,18.00,6035,CA,County,30573,17777.12,58.88,58.88,17963.56,34,1,0.00,1.00
3284,02/07/21,Lassen,California,5435,18.00,6035,CA,County,30573,17777.12,58.88,58.88,17963.56,34,1,0.00,0.00
3285,02/08/21,Lassen,California,5489,18.00,6035,CA,County,30573,17953.75,58.88,58.88,17963.56,34,1,54.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,315.65,14637.86,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,315.65,14637.86,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,315.65,14637.86,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,315.65,14637.86,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,315.65,14637.86,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356,02/05/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,121.89,8449.28,10,18,361.00,15.00
3357,02/06/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,121.89,8449.28,10,18,0.00,0.00
3358,02/07/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,121.89,8449.28,10,18,0.00,0.00
3359,02/08/21,San Joaquin,California,64138,923.00,6077,CA,County,762148,8415.43,121.11,121.89,8449.28,10,18,519.00,30.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)